In [1]:
import altair as alt


def make_bars_chart(df, title, scale_title=None, avg_col='avg', sqrt_scale=True,
                   width=50, height=300, multi_bar=True, legend=True, labels=True):
    source = df.copy()

    if source[avg_col].max() > 1:
        avg_scale = 'secs'
        avg_coef = 1
    elif source[avg_col].max() > 0.001:
        avg_scale = 'ms'
        avg_coef = 1e3
    else:
        avg_scale = 'µs'
        avg_coef = 1e6

    source[avg_scale] = (df[avg_col] * avg_coef).round(2)
    
    if not scale_title:
        scale_titles = {
            'secs': 'seconds',
            'ms': 'milliseconds (1e−3 secs)',
            'µs': 'microseconds (1e−6 secs)',
        }
        scale_title = scale_titles[avg_scale]

    if sqrt_scale:
        y_scale = alt.Scale(type='sqrt')
    else:
        y_scale = alt.Scale()
    
    if multi_bar:
        x_val = 'bench:N'
        facet_kwds = {'column':'name:N'}
    else:
        x_val = 'name:N'
        facet_kwds = {}

    if legend:
        legend = alt.Legend()
    else:
        legend = None

    chart = alt.Chart(
        width=width,
        height=height,
    ).mark_bar(
        stroke='transparent',
        size=20,
    ).encode(
        alt.X(x_val, scale=alt.Scale(), axis=alt.Axis(title='', labels=labels)),
        alt.Y(f'{avg_scale}:Q', scale=y_scale, axis=alt.Axis(title=scale_title, grid=False)),
        color=alt.Color(x_val, scale=alt.Scale(range=["#FF7B06", "#094AFB", "#D60000"]), legend=legend),
    )

    text = chart.mark_text(
        color='black',
        dx = 0,
        dy = -2,
    ).encode(
        text=f'{avg_scale}:Q'
    )

    return alt.layer(chart, text, data=source).facet(
        **facet_kwds
    ).configure_axis(
        domainWidth=0.8,
    ).configure_view(
        stroke='transparent'
    ).properties(
        title=title
    )

In [2]:
import pandas as pd
single_df = pd.read_json('bench_results.json', orient='records')
single_df.head()

,bench,max,mean,min,std,tool
0,1-8digits,0.579355,0.556693,0.538694,0.012929,cracken
1,1-8digits,0.715639,0.687277,0.667732,0.011805,maskprocessor
2,9digits,5.045765,4.826192,4.673063,0.110049,cracken
3,9digits,184.738300,184.738300,184.738300,NaN,crunch
4,9digits,6.241444,6.001537,5.862338,0.101060,maskprocessor


In [3]:
df = single_df.groupby(['bench', 'tool']).mean().reset_index()
df['tool'].replace('crunch', ' crunch', inplace=True)
df

,bench,tool,max,mean,min,std
0,1-8digits,cracken,0.579355,0.556693,0.538694,0.012929
1,1-8digits,maskprocessor,0.715639,0.687277,0.667732,0.011805
2,9digits,cracken,5.045765,4.826192,4.673063,0.110049
3,9digits,crunch,184.738300,184.738300,184.738300,NaN
4,9digits,maskprocessor,6.241444,6.001537,5.862338,0.101060
5,upper-5lower-digit,cracken,15.212566,14.847242,14.769393,0.138524
6,upper-5lower-digit,crunch,474.109697,474.109697,474.109697,NaN
7,upper-5lower-digit,maskprocessor,19.390088,19.144515,18.525928,0.293365


In [4]:
df['secs'] = df['mean']
df['name'] = df['tool']
df['ms'] = df['mean'] * 1000.0
df['µs'] = df['mean'] * 1_000_000.0
df

,bench,tool,max,mean,min,std,secs,name,ms,µs
0,1-8digits,cracken,0.579355,0.556693,0.538694,0.012929,0.556693,cracken,556.692714,5.566927e+05
1,1-8digits,maskprocessor,0.715639,0.687277,0.667732,0.011805,0.687277,maskprocessor,687.276880,6.872769e+05
2,9digits,cracken,5.045765,4.826192,4.673063,0.110049,4.826192,cracken,4826.191874,4.826192e+06
3,9digits,crunch,184.738300,184.738300,184.738300,NaN,184.738300,crunch,184738.299608,1.847383e+08
4,9digits,maskprocessor,6.241444,6.001537,5.862338,0.101060,6.001537,maskprocessor,6001.537437,6.001537e+06
5,upper-5lower-digit,cracken,15.212566,14.847242,14.769393,0.138524,14.847242,cracken,14847.241614,1.484724e+07
6,upper-5lower-digit,crunch,474.109697,474.109697,474.109697,NaN,474.109697,crunch,474109.697342,4.741097e+08
7,upper-5lower-digit,maskprocessor,19.390088,19.144515,18.525928,0.293365,19.144515,maskprocessor,19144.515344,1.914452e+07


In [5]:
make_bars_chart(
    df,
    title='Wordlist Generation Time',
    sqrt_scale=True,
    labels=False,
    avg_col='mean',
    width=150,
    height=300,
)

alt.FacetChart(...)

In [6]:
min_took_df = df[df['tool'] == 'cracken'][['bench', 'mean']].groupby('bench').min().reset_index()
min_took = {x.bench: x.mean for x in min_took_df.itertuples()}
min_took

{'1-8digits': 0.5566927139405851,
 '9digits': 4.826191873550415,
 'upper-5lower-digit': 14.847241613599989}

In [7]:
df['speedup'] = df.apply(lambda x: x['mean'] / min_took[x.bench], axis=1)

In [8]:
make_bars_chart(
    df,
    title='Wordlist Generation Time Speedup',
    sqrt_scale=True,
    labels=False,
    avg_col='speedup',
    width=150,
    height=300,
)

alt.FacetChart(...)

In [9]:
df.groupby('tool').mean()[['speedup']]

,speedup
tool,
crunch,35.105393
cracken,1.000000
maskprocessor,1.255846
